# Iron-supported Graphene ORR Free Energy Diagram

### ==== Import Modules ====

In [1]:
# import sys
import copy
# import traceback

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import plotly
from plotly.graph_objs import Figure
plotly.tools.set_credentials_file(username='flores12', api_key='Yhmt9Yq57vXUcxQmRMmY')

# My Modules ******************************************************************
from dft_job_automat.job_analysis import DFT_Jobs_Analysis
from dft_job_automat.job_types_classes.dft_methods import DFT_Methods
from orr_reaction.orr_methods import ORR_Free_E_Plot, calc_ads_e

from orr_reaction.orr_methods import lowest_e_path, df_calc_adsorption_e, plot_all_states

### ==== Script Inputs ====

In [2]:
# Colors
color_list = [
    "rgb(37,69,0)",
    "rgb(0,76,214)",
    "rgb(168,0,15)",
    "rgb(0,85,162)",
    "rgb(255,113,196)",
    ]

color_list = [
    "rgb(113,166,190)",
    "rgb(145,209,79)",
    "rgb(124,78,196)",
    "rgb(203,169,87)",
    "rgb(200,72,150)",
    "rgb(130,204,159)",
    "rgb(190,82,63)",
    "rgb(80,51,82)",
    "rgb(81,92,54)",
    "rgb(192,151,188)",
    ]

fe_corr_dict = {
    "h2o": 0.5290403581846084,
    "o": 0.0777295188024034,
    "oh": 0.3278711654655712,
    "ooh": 0.35756420469723127,
    }

# Free Energy Corrections
# COMBAK I'm doing it manually for now
#fe_corr_dict = {
#    "h2o": 0.,
#    "ooh": 0.,
#    # "o": 0.,
#    "o": 0.01,
#    # "oh": 0.,
#    "oh": -0.25,
#    }

run_an_0 = True
run_an_1 = True

ref_scheme = "h2o"  # o2 or h2o

bias = 0.

### ==== Reference Energies ====

In [3]:

#| - Slab Energy
# /scratch/users/flores12/03_graph_N_Fe/01_opt_struct/
# combined-Fe-graph/FCC/5-att/1STEP/data/03-800/03-881/01-True/_1

# /scratch/users/flores12/03_graph_N_Fe/01_opt_struct/
#  combined-Fe-graph/FCC/5-att/1STEP/data/03-800/03-881/02-False/_1

bare_slab = -31722.43520417364
bare_slab_SP_t = -31722.43520417364
bare_slab_SP_f = -31721.412152900266

bare_slab_dict = {
    False: bare_slab_SP_f,
    True: bare_slab_SP_t,
    }
#__|

# R21 From Pourbaix Papers
dG_rxn_h2o = -2.4583

# /scratch/users/flores12/gas_phase_molec/BEEF-vdW
H2_dft = -32.9563981542851
O2_dft = -883.190570481887
H2O_dft = -476.544109028354

hyd_ref = H2_dft / 2.

if ref_scheme == "h2o":
    oxy_ref = H2O_dft - H2_dft
elif ref_scheme == "o2":
    oxy_ref = dG_rxn_h2o + O2_dft / 2.

### ==== Initiate Instance ====

In [4]:
dft_inst = DFT_Methods(
    methods_to_run=[
        "elec_energy",
        "atom_type_num_dict",
        "init_atoms",
        "atoms_object",
        "magmom_charge_history",
        ],
    )

Jobs = DFT_Jobs_Analysis(
    update_job_state=False,
    job_type_class=dft_inst,
    # load_dataframe=True,
    load_dataframe=True,
    working_dir="1STEP",
    )

df = Jobs.filter_early_revisions(Jobs.data_frame)

df_master = df.copy()

# from dft_job_automat.job_types_classes.data_frame_methods import \
#     DataFrame_Methods
# DF = DataFrame_Methods(df)
# DF.create_atoms_objects(atoms_row="atoms_object")


### ==== Data Processing ====

#### == Filtering Data ==

In [5]:
# tmp = df[np.isfinite(df["elec_energy"]) == True]
# tmp = df[np.isfinite(df["elec_energy"]) == True]
# # df = df[df["job_state"] == "SUCCEEDED"]
#
# finite_e_rows = np.isfinite(df["elec_energy"]) == True
# df = df[finite_e_rows]



# df = df[df["spinpol"] == True]
# df = df[df["notes"] != "Desorbed"]

#### == Removing Unnecessary Columns ==

In [6]:
col_to_remove = [
    "variable_list",
    "init_atoms",
    "atoms_object",
    ]

df = df.drop(col_to_remove, axis=1)

#### == Adsorption Energies ==

In [7]:
ads_e_list = []
for index, row in df.iterrows():

    corr = fe_corr_dict[row["adsorbate"]]
    bare_e = bare_slab_dict[row["spinpol"]]

    ads_e_i = calc_ads_e(
        row,
        # bare_slab,
        bare_e,
        correction=corr,
        oxy_ref_e=oxy_ref,
        hyd_ref_e=hyd_ref,
        )

    ads_e_list.append(ads_e_i)

df["ads_e"] = np.array(ads_e_list)
df_master = copy.deepcopy(df)

### ==== Data Analysis ====


#### === Analysis 1 (All States Plotted) ===

In [9]:
dat_lst1, layout1 = plot_all_states(df_master, Jobs.tree_level_labels, color_list, bias=0.)

import plotly.plotly as py
py.iplot(
    {
        "data": dat_lst1,
        "layout": layout1,
        },
    filename="pl_fed_supp_graph_01.html"
    )

#### == Analysis 2 (Minimum Energy States) ==

In [ ]:
dat_lst, layout = lowest_e_path(df_master, Jobs.tree_level_labels, color_list, bias=0.)

py.iplot(
    {
        "data": dat_lst,
        "layout": layout,
        },
    filename="pl_fed_supp_graph_02.html"
    )

#### Analysis 3 (Spin State Configurations)

In [ ]:
def proc_df_i(df, elem="Fe"):
    """
    """
    #| - proc_df_i
#    df = df_mm
#    elem = "Fe"

    max_iter = df["iteration"].max()

    df = df[df["iteration"] == max_iter]
    df = df[df["element"] == elem]

    # print(df)
    return(df)
    #__|
#df["magmom_history"].iloc[0]

import numpy as np

df_list = []
for index, row in df.iterrows():
    df_mm = row["magmom_history"]

    if isinstance(df_mm, pd.DataFrame):
        df_out_i = proc_df_i(df_mm)
        df_list.append(df_out_i)

df_new = pd.concat(df_list)

import cufflinks as cf

df_new.iplot(kind="scatter", mode="markers", x="atom_num", y="magmom")